# Installing Dependencies

In [1]:
using DelimitedFiles
using NearestNeighbors: KDTree, knn
using LinearAlgebra: norm

## Using General Arrays

In [2]:
function NearNeighbor(X, t, t_knn)
    # Using kNN for Nearest Neighbors
    if t%t_knn | t == 0
        global kdtree = KDTree(X[:,1:3]')
    end 
    return kdtree
end

function force(X, r_max, s, K, kdtree)
    # Initialise displacement array
    global dX = zeros(Float64, size(X)[1], 3)

    # Loop over all cells to compute displacements
    for i in 1:size(X)[1]
        # Scan neighbours
        global idxs, _ = knn(kdtree, X[i,1:3], 14, true)

        # Initialise variables
        global Xi = X[i,1:3]
        for j in 1:size(X)[1]
            if i != j
                global r = Xi - X[j,1:3]
                global dist = norm(r)
                # Calculate attraction/repulsion force differential here
                if dist < r_max
                    global F = - K*(dist-r_max)*(dist-r_max)*(dist - s)
                    dX[i,:] +=  r/dist * F
                end
            end
        end
    end
    return dX
end

force (generic function with 1 method)

In [3]:
R_Agg = 16

X = readdlm("../data/Init/Two_Sphere/$R_Agg.csv", ',', Float64, header=true)[1][:, 1:3]
r_max, s = 2, 1
K = 10
t, t_knn = 0, 10
dt = 0.5;

In [5]:
@time kdtree = KDTree(X[:,1:3]')

  0.000929 seconds (33 allocations: 541.422 KiB)


KDTree{StaticArraysCore.SVector{3, Float64}, Distances.Euclidean, Float64}
  Number of points: 6064
  Dimensions: 3
  Metric: Distances.Euclidean(0.0)
  Reordered: true

In [7]:
@time kd = NearNeighbor(X, t, t_knn)

  0.000982 seconds (30 allocations: 541.344 KiB)


KDTree{StaticArraysCore.SVector{3, Float64}, Distances.Euclidean, Float64}
  Number of points: 6064
  Dimensions: 3
  Metric: Distances.Euclidean(0.0)
  Reordered: true

In [ ]:
@time F = force(X, r_max, s, K, kd)

## Using kNN

In [8]:
function NearNeighbor(X, t, t_knn)
    # Using kNN for Nearest Neighbors
    if t%t_knn | t == 0
        global kdtree = KDTree(X[:,1:3]')
    end 
    return kdtree
end

function force(X, r_max, s, K, kdtree)
    # Initialise displacement array
    global dX = zeros(Float64, size(X)[1], 3)

    # Loop over all cells to compute displacements
    for i in 1:size(X)[1]
        # Scan neighbours
        global idxs, _ = knn(kdtree, X[i,1:3], 14, true)

        # Initialise variables
        global Xi = X[i,1:3]
        for j in idxs
            if i != j
                global r = Xi - X[j,1:3]
                global dist = norm(r)
                # Calculate attraction/repulsion force differential here
                if dist < r_max
                    global F = - K*(dist-r_max)*(dist-r_max)*(dist - s)
                    dX[i,:] +=  r/dist * F
                end
            end
        end
    end
    return dX
end

R_Agg = 16

X = readdlm("../data/Init/Two_Sphere/$R_Agg.csv", ',', Float64, header=true)[1][:, 1:3]
r_max, s = 2, 1
K = 10
t, t_knn = 0, 10
dt = 0.5

0.5

In [24]:
@time kd = NearNeighbor(X, t, t_knn)

  0.007504 seconds (14.25 k allocations: 1.328 MiB, 87.79% compilation time)


KDTree{StaticArraysCore.SVector{3, Float64}, Distances.Euclidean, Float64}
  Number of points: 6064
  Dimensions: 3
  Metric: Distances.Euclidean(0.0)
  Reordered: true

In [28]:
@time F = force(X, r_max, s, K, kd)

  0.056425 seconds (1.53 M allocations: 50.659 MiB, 5.72% gc time)


6064×3 Matrix{Float64}:
  0.0         -0.00013623    0.000261888
  0.0         -0.00013623    0.000261888
  0.0         -0.00013623    0.000261888
  0.0          0.000172509   0.000337449
  0.0          0.000172509   0.000337449
  0.0          0.000172509   0.000337449
 -1.57531e-5   0.000145256   0.000337449
  3.15062e-5   0.000118003   0.000337449
  0.0          0.000118003   0.000337449
  0.0          0.000118003   0.000337449
  ⋮                         
  0.0         -0.000118003  -0.000337449
 -1.57531e-5  -9.07506e-5   -0.000337449
  0.0         -0.000172509  -0.000337449
  0.0         -0.000172509  -0.000337449
  0.0         -0.000172509  -0.000337449
 -1.57531e-5  -0.000145256  -0.000337449
  0.0          0.00013623   -0.000261888
  0.0          0.00013623   -0.000261888
  0.0          0.00013623   -0.000261888